In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [24]:
model_save_path = '/content/drive/MyDrive/animal_model.keras'
cnn_model = load_model(model_save_path)
cnn_model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,610,314 (25.22 MB)

 Trainable params: 3,305,156 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,305,158 (12.61 MB)

In [26]:
input_shape = (128, 128, 3)
new_input = tf.keras.Input(shape=input_shape)
model_output = cnn_model.get_layer("flatten").output
defined_input_model = Model(inputs=new_input, outputs=model_output)

In [28]:
feature_extractor_model = Model(inputs=cnn_model.inputs, outputs=cnn_model.get_layer("flatten").output)
# feature_extractor_model = Model(inputs=cnn_model.input, outputs=cnn_model.get_layer("flatten").output)

In [29]:
def extract_features(generator):
    generator.reset()
    features = feature_extractor_model.predict(generator, steps=generator.samples // generator.batch_size + 1)

    labels = generator.classes[:len(features)]
    return features, labels

In [31]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [32]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

In [34]:
dataset_path = '/content/drive/MyDrive/animal_data'

In [35]:
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical',
    subset='training',
    shuffle=False
)

Found 3677 images belonging to 4 classes.


In [36]:
validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 916 images belonging to 4 classes.


In [37]:
train_features, train_labels = extract_features(train_generator)
validation_features, validation_labels = extract_features(validation_generator)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


230/230 ━━━━━━━━━━━━━━━━━━━━ 777s 3s/step
58/58 ━━━━━━━━━━━━━━━━━━━━ 246s 4s/step


In [38]:
rf_model = RandomForestClassifier(n_estimators=50, random_state=42)
rf_model.fit(train_features, train_labels)

RandomForestClassifier(n_estimators=50, random_state=42)

In [39]:
validation_predictions = rf_model.predict(validation_features)

In [40]:
print('Classification Report')
print(classification_report(validation_labels, validation_predictions, target_names=validation_generator.class_indices.keys()))

Classification Report
              precision    recall  f1-score   support

         cat       0.54      0.62      0.57       237
         dog       0.51      0.35      0.42       237
    elephant       0.64      0.66      0.65       205
       tiger       0.72      0.83      0.77       237

    accuracy                           0.61       916
   macro avg       0.60      0.61      0.60       916
weighted avg       0.60      0.61      0.60       916



In [41]:
print('Confusion Matrix')
print(confusion_matrix(validation_labels, validation_predictions))

Confusion Matrix
[[146  57  16  18]
 [ 89  84  38  26]
 [ 22  17 135  31]
 [ 14   6  21 196]]
